# 第6回：予測モデル作成
* 前回まででデータ理解やクレンジングができたので、いよいよ機械学習で予測モデルを作成していきましょう。
* 機械学習のアルゴリズムは色々とありますが、データサイエンス総論Ⅱの演習では決定木、ランダムフォレストと呼ばれる手法を使います。
* 今回は実際に予測モデルを作り、精度を評価してみましょう。

# 1. データ読み込み
* まずはデータを読み込みましょう。
* 読み込む方法は前回までと同様ですが、今回は前回整形したデータ（data_clensing.csv）を読み込みます。

In [ ]:
import pandas as pd             # 表形式のデータ操作ライブラリ
import matplotlib.pyplot as plt # グラフ描画ライブラリ
import seaborn as sns           # グラフ描画ライブラリ

from sklearn.tree import DecisionTreeClassifier      # 決定木による分類のためのライブラリ
from sklearn.ensemble import RandomForestClassifier  # ランダムフォレストによる分類のためのライブラリ

from sklearn.metrics import roc_auc_score     # AUC値を算出するライブラリ
from sklearn.metrics import roc_curve         # ROC曲線を描画するためのライブラリ
from sklearn.metrics import accuracy_score    # 正解率を算出するライブラリ
from sklearn.metrics import confusion_matrix  # 混同行列を作成するライブラリ

random_state = 1 # 乱数固定
sns.set() 

In [ ]:
# 警告（warning）を表示させない
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 前回の演習でクレンジング・特徴量作成済みのデータを読み込みます
data = pd.read_csv('data_clensing.csv')

In [ ]:
print('データのサイズ：', data.shape) # データサイズ確認
display(data.head())                  # データが正しく読み込めたことを確認

# 2. 各カテゴリ変数を数値に変換
* 本講義では、予測のための機械学習にsklearnという機械学習用のライブラリを活用します。
* sklearnで予測モデルを学習できるようにするためには、"yes"や"no"などの文字で表現されているデータを数値で表す必要があります。
* 各カテゴリ変数について数値化していきましょう。

## 2-1. 目的変数を数値に変換
* •sklearnで予測モデルを学習できるように、目的変数（y列）を"yes"を1、"no"を0に変換して、1と0の数値で表すことにします。

In [ ]:
data['y'] = data['y'].map( {'yes': 1, 'no': 0} ) # yesを1, noを0に変換

In [ ]:
data['y'].head() # 変換後のデータを表示し、数値に変換できたかを確認

In [ ]:
data['y'].value_counts()

## 2-2. その他のカテゴリ変数を数値に変換
* y, month以外のカテゴリ変数を一括で変換していきます。
* ここでは、One-Hotエンコーディングを呼ばれる数値化手法を使います。
* たとえばローンの有無を表すloan 列には"yes"と"no"の2種類ありますが、新たに loan_yes列とloan_no列を作成し、loanがyesのデータはloan_yesが1、loan_noが0になるように変換します。

* 複雑な処理が必要になるように思えますが、pandasのget_dummiesという機能を使うことで簡単に変換できます。

In [ ]:
# まずは変換対象のカテゴリ列をまとめて categorical_columns という名前で保存します
categorical_columns =  ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']

# get_dummies機能を使って、categorical_columns に保存した列を対象にOne-Hotエンコーディングを行います。
data = pd.get_dummies(data, columns = categorical_columns)

In [ ]:
data.head() # データを表示し、正しく変換できたことを確認

* "loan"列を"loan_no"列と"loan_yes"列でOne-Hot表現できています。ほかのカテゴリ変数も正しく数値に変換できたことが確認できました。

## 2-3.データの保存
* 機械学習に向けてのデータ数値化が完了したので、今後同じ処理をせずに済むようにCSVファイルとして保存しておきましょう。

In [ ]:
data.to_csv('data_ml.csv', index=False)

# 3. 予測モデルの作成
* データの準備が終わったので、いよいよ予測モデルを作っていきましょう。
* 今回は機械学習の手法（アルゴリズム）として**決定木**と呼ばれる手法と**ランダムフォレスト**と呼ばれる手法を使います。
+ 決定木は、データから「顧客が40歳以上か40歳未満か」「住宅ローンがあるか無いか」のような傾向を発見することで、  
「顧客が40代以上で住宅ローンがないと定期預金をする」というようなパターンに基づく予測ができるようになります。
* ランダムフォレストは、決定木を多数作り予測結果の多数決をとることで精度向上を目指す手法です。
* まずは、決定木で予測モデル作成の流れを体験しましょう。

## 3-1. 学習・予測
* データを説明変数と目的変数に分割し、予測モデル作成を行います。
* 説明変数をX_train, 目的変数をy_trainとして予測モデルを学習し、X_trainに対する予測精度を確認してみましょう。

In [ ]:
X_train = data.drop(['y'], axis=1)
y_train = data['y']

### 決定木の予測モデル作成

In [ ]:
# モデルの定義
tree = DecisionTreeClassifier()     # この時点でtreeは未学習
# 学習
tree.fit(X_train, y_train)          # fit関数でtreeが学習データをもとに分岐を作り学習を行う
# 予測
y_pred_tree = tree.predict(X_train) # 学習済みのtreeを使って予測を行う

## 3-2. 精度評価
* 予測モデルが学習できたので、精度を評価してみましょう。
* 精度評価の指標はいろいろとありますが、今回は正解率とAUCで評価してみます。

### 正解率での評価（全データ数のうち、予測が当たったデータの割合）

In [ ]:
ac_tree = accuracy_score(y_train, y_pred_tree)

print("決定木の正解率：", ac_tree)

### 混同行列の表示

In [ ]:
cm_tree = confusion_matrix(y_train, y_pred_tree)

print("---決定木の混同行列---")
print(cm_tree)

### ROCカーブの表示

In [ ]:
y_pred_tree = tree.predict_proba(X_train) # 0,1ではなく予測確率を出力

fpr, tpr, thresholds = roc_curve(y_train, y_pred_tree[:,1])
plt.figure(figsize=(5,5))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],color="gray",linestyle="dotted")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

### ROC AUCでの評価

In [ ]:
auc_tree = roc_auc_score(y_train, y_pred_tree[:,1])
print("決定木のAUC：", auc_tree)

* AUCの値はビジネスケースやデータの形式にかかわらず同じ尺度で比較できるのがメリットです
    * 感覚論ですが一般的に0.7以上ならある程度予測ができており、0.9以上ならかなり精度の高い予測ができているといえます。